Assignment 4:

Name:

UID:

Please submit to ELMS
- a PDF containing all outputs (by executing **Run all**)
- your ipynb notebook containing all the code

I understand the policy on academic integraty (collaboration and the use of online material).
Please sign your name here:

In [ ]:
## Import necessary libraries here (You can add libraries you want to use here)
import os
import random
import cv2
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

# Part 1: Epipolar Geometry (30 Points)


## Overview

In this problem, you will implement an algorithm for automatically estimating homography with RANSAC. In the file matches.mat, we provide the detected Harris corners row-column positions in variables r1 c1 for the first image; variables r2 c2 for the second image; and the corresponding matched pairs in the variable matches.

<!-- <img src="https://drive.google.com/uc?id=1Tr723u5OXmwkd4RDmu9z886ITJU9j1cL&export=download" width="800"/> -->

<img src="https://www.cs.umd.edu/class/spring2023/cmsc426-0201/hw_images/epipolar_geometry.jpg" width="800"/>


The outline of the normalized 8-point algorithm:

<img src="https://www.cs.umd.edu/class/spring2023/cmsc426-0201/hw_images/8pointalgo.jpg" width="700"/>



## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_
# Or download from https://drive.google.com/file/d/1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_/view and upload

!unzip "/content/Part1_data.zip" -d "/content/"
# Load Matches
data = loadmat('/content/Part1_data/matches.mat')
r1 = data['r1']
r2 = data['r2']
c1 = data['c1']
c2 = data['c2']
matches = data['matches']

Downloading...
From: https://drive.google.com/uc?id=1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_
To: /content/Part1_data.zip
100% 157k/157k [00:00<00:00, 68.8MB/s]
Archive:  /content/Part1_data.zip
   creating: /content/Part1_data/
  inflating: /content/Part1_data/chapel00.png  
  inflating: /content/Part1_data/chapel01.png  
  inflating: /content/Part1_data/matches.mat  


In [ ]:
# Load Keypoints
x1 = c1[matches[:,0]-1]
y1 = r1[matches[:,0]-1]
x2 = c2[matches[:,1]-1]
y2 = r2[matches[:,1]-1]

## Helper Functions

In [ ]:
from scipy.ndimage import gaussian_filter as gf
from google.colab.patches import cv2_imshow

def show_image(img, scale=1.0):
    plt.figure(figsize=scale* plt.figaspect(1))
    plt.imshow(img, interpolation='nearest')
    plt.gray()
    plt.axis('off')
    plt.show()


### Code (15 pt)

### Hint

*   For visualization, you can use cv2.line, cv2.circle or any other helper functions in opencv or matplotlib.


In [ ]:
def normalize(x, y):
  """
  Function: find the transformation T to make coordinates zero mean and the variance as sqrt(2)
  Input: x, y - coordinates
  Output: normalized coordinates, transformation T
  """
  # YOUR CODE HERE:

  return None

def ransacF(x1, y1, x2, y2, num_iterations=1000, threshold=0.01):
  """
  Find normalization matrix
  Transform point set 1 and 2
  RANSAC based 8-point algorithm
  Input:
     x1, y1, x2, y2 - coordinates
     num_iterations - how many iterations
     threshold - threshold for inlier check
  Output:
     Best fundamental matrix
     corresponding inlier indices
  """
  # YOUR CODE HERE:

  # Hint:
  # for ... in num_iterations:
  #    1. Randomly select 8 points

  #    2. Call computeF()

  #    3. Call getInliers()

  #    4. Update F and inliers.

  return None

def computeF(x1, y1, x2, y2):
  """
  Function: compute fundamental matrix from corresponding points
  Input:
     x1, y1, x2, y2 - coordinates
  Output:
     fundamental matrix, 3x3
  """
  # YOUR CODE HERE:

  # Hint:
  # 1. Make matrix A

  # 2. Do SVD for A

  # 3. Find fundamental matrix F

  return None

def getInliers(x1, y1, x2, y2, F, thresh):
  """
   Function: implement the criteria checking inliers.
   Input:
     x1, y1, x2, y2 - coordinates
     F - estimated fundamental matrix, 3x3
     thresh - threshold for passing the error
   Output:
     inlier indices
  """
  # YOUR CODE HERE:

  # Hints:
  # Say we project the points onto the second image
  # 1. Compute epipolar lines. Here a line is expressed as a vector
  # Recall line equation: ax + by + c = 0

  # 2. Calculate the distances of points in the second image from the corresponding lines
  # Recall that the distance between a point and a line is
  # d = |ax + by + c| / sqrt(a^2 + b^2)

  # 3. Check distances with the threshold and find inliers

  return None


In [ ]:
# No rush, let's do a sanity check for computeF()
from sklearn.metrics import mean_squared_error

# A synthetic dataset with matched points
p1 = np.array([[83, 245], [39, 461], [284, 243],[328, 459],   # points of image1 in 2D coordinates
              [331, 166], [286, 47], [291, 332],[296, 303]])
p2 = np.array([[36, 155], [34, 625], [329, 158],[323, 627],   # points of image2 in 2D coordinates
              [375, 714], [375, 67], [313, 398],[327, 331]])

px1, py1 = p1[:, :1], p1[:, 1:2]
px2, py2 = p2[:, :1], p2[:, 1:2]

# Normalize points
norm_p1, T1 = normalize(px1, py1)
norm_x1, norm_y1 = norm_p1[:, :1], norm_p1[:, 1:2]
norm_p2, T2 = normalize(px2, py2)
norm_x2, norm_y2 = norm_p2[:, :1], norm_p2[:, 1:2]

# Compute fundamental matrix with computeF()
F_ours = computeF(norm_x1, norm_y1, norm_x2, norm_y2)
F_ours = np.matmul(np.matmul(np.transpose(T2),F_ours), T1)
F_ours = F_ours / F_ours[2, 2]
print("Our F: ", F_ours)
# Compute fundamental matrix with cv2.findFundamentalMat()
F_cv2, _ = cv2.findFundamentalMat(p1, p2, method=cv2.FM_8POINT)
print("CV2 F: ", F_cv2)

# Display difference, expected to be very small.
mse_err = mean_squared_error(F_ours, F_cv2)
print(f'\n MSE of fundamental matrixes = {mse_err }')

if mse_err < 1e-12:
  print("Sanity check passed!")
else:
  print("Sanify check failed!")

In [ ]:
# Using the functions implemented above, write the CODE HERE to get the write up
# estimate the fundamental matrix F


# 1. Normalize coordinates. Call `normalize()` above.


# 2. Run 8-point algorithm with RANSAC to estimate F. Run `ransacF()`.
# You can decide num_iterations by computing
# log(1 - p)/log(1 - (1 - e)**s), check Lecture 10.


# 3. De-normalize F using T, T' from 1.


# 4. Normalize F so that the last entry is 1.
# NOTE: this normalization is different from what the normalize() function above does
# this is just a simple normalization that scales the matrix so the last entry is 1


# 5. Report normalized F from 4.


### Write-up (15 pt)
*   Describe what test you used for deciding inlier vs. outlier.
*   Print the estimated fundamental matrix F after normalizing to unit length
*   Randomly select 7 sets of matching points. Plot the corresponding epipolar lines and the points on each image. Show the two images (with plotted points and lines) next to each other.

<!-- *   Plot the outlier keypoints with green dots on top of the first image -->
<!-- *   Randomly select 7 sets of matching points. Plot the corresponding epipolar lines ('g’) and the points (with 'r+’) on each image. Show the two images (with plotted points and lines) next to each other. -->



In [ ]:
# plot the images with the corresponding epipolar lines
# you may use any plotting library or tool you like

# A possible way:
# 1. Randomly select 7 pairs of points from two images. You can use `np.random.choice()`.

# 2. Compute epipolar lines for selected points.
# Recall how we represent a line above

# 3. Draw lines and points for selected points
# See example in Overview above.
# You can use cv2.line() and cv2.circle().

# 4. Show two images side by side.
# You can use `np.concatenate()`.


# Part 2: Image stitching (30 points)

<img src="https://www.cs.umd.edu/class/spring2023/cmsc426-0201/hw_images/image_stitching.png" width="800"/>

## Overview

In this problem, you will implement an algorithm for automatically estimating the fundamental matrix F using RANSAC and the normalized 8-point algorithm.

Image Stitching Algorithm Overview
1. Detect keypoints
2. Match keypoints
3. Estimate homography with matched keypoints (using RANSAC)
4. Combine images

**Note:**  Do not use existing image stitching code, such as found on the web, and OpenCV.

## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 1fnD0hJ8-_Rngsc-m96ghKtdZAMf0VTjy
!unzip "/content/hill.zip" -d "/content/hill"

!gdown 1v2BFVMV0McuD5BstLvDmo1U9MrFAByS5
!unzip "/content/tv.zip" -d "/content/tv"


Downloading...
From: https://drive.google.com/uc?id=1fnD0hJ8-_Rngsc-m96ghKtdZAMf0VTjy
To: /content/hill.zip
100% 205k/205k [00:00<00:00, 6.39MB/s]
Archive:  /content/hill.zip
  inflating: /content/hill/1.JPG     
  inflating: /content/hill/2.JPG     
  inflating: /content/hill/3.JPG     
Downloading...
From: https://drive.google.com/uc?id=1v2BFVMV0McuD5BstLvDmo1U9MrFAByS5
To: /content/tv.zip
100% 130k/130k [00:00<00:00, 14.8MB/s]
Archive:  /content/tv.zip
  inflating: /content/tv/1.jpg       
  inflating: /content/tv/2.jpg       
  inflating: /content/tv/3.jpg       


### Hint


*   Use SIFT to find keypoint. You can use sift.detectAndCompute in opencv.
*   For image warping and blending, you should first deterimne canvas size. You can use cv2.warpPerspective in opencv.

An example is as follows:

<img src="https://www.cs.umd.edu/class/spring2024/cmsc426-0101/img/stitch.jpg" width="800"/>




## Helper Functions

In [ ]:
import cv2

def drawMatches(image1, kp1, image2, kp2, idx_pairs):
    keypt1 = [cv2.KeyPoint(coord[1], coord[0], 40) for coord in kp1.tolist()]
    keypt2 = [cv2.KeyPoint(coord[1], coord[0], 40) for coord in kp2.tolist()]
    matches = [cv2.DMatch(pair[0], pair[1], 0) for pair in idx_pairs.tolist()]
    return cv2.drawMatches(image1, keypt1, image2, keypt2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

def plot_matches(images, feature_coord, matches, img_idx):
    matched_img = drawMatches(images[img_idx], feature_coord[img_idx], images[img_idx-1],
                              feature_coord[img_idx-1], matches[img_idx-1])

    cv2.imshow('Matches Found', matched_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def est_homography(src, dest):
    N = src.shape[0]
    if N != dest.shape[0]:
        raise ValueError("src and diff should have the same dimension")
    src_h = np.hstack((src, np.ones((N, 1))))
    A = np.array([np.block([[src_h[n], np.zeros(3), -dest[n, 0] * src_h[n]],
                            [np.zeros(3), src_h[n], -dest[n, 1] * src_h[n]]])
                  for n in range(N)]).reshape(2 * N, 9)
    [_, _, V] = np.linalg.svd(A)
    return V.T[:, 8].reshape(3, 3)

def apply_homography(H, src):
    src_h = np.hstack((src, np.ones((src.shape[0], 1))))
    dest =  src_h @ H.T
    return (dest / dest[:,[2]])[:,0:2]

### Code (15 pt)

In [ ]:
# YOUR CODE HERE:
def crop_dark(panorama):
    gray_panorama = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_panorama, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_area_contour)
    panorama = panorama[y:y+h, x:x+w]

    return panorama


def find_homography_ransac(src, dest, threshold=0.5, max_iterations=1000):
    """
    Run RANSAC to estimate H.
    Input:
    - src, dest: coordinates
    - threshold: threshold for RANSAC
    - max_iterations: number of iterations for RANSAC
    Output:
    - estimated homography H
    - inliers
    """

    pass


def stitch_images(img1, img2):
    """
    Stitch two images at a time.
    Inputs:
    - img1, img2 with shape [H, W, 3].
    Output:
    - stitched_image with shape [H, W, 3].
    """
    #1. Detect keypoints
    # SIFT feature detection and matching
    # You can use sift.detectAndCompute() from cv2


    #2. Match keypoints
    # hint: You will need to create descriptors. One way to do that is to use
    # sift. You can use SIFT from opencv (cv2) for that
    # or you can use your preferred method of creating descriptors
    # KNN can be useful to match points, use e.g., cv2.FlannBasedMatcher()
    # You can further filter out some matches by using a threshold.



    #3. Estimate homography with matched keypoints (using RANSAC)
    # Call `find_homography_ransac()`



    #4. Combine images
    # Here we assume all the images are ordered, from left to right: "1.jpg,
    # 2.jpg, 3.jpg...". You can use `cv2.warpPerspective()` to warp images.
    # Be careful: do we use forward warping or inverse warping?
    # You probably want to crop out some dark part. Call crop_dark() above.


In [ ]:
# Let's first try two images.
import glob
case_name = "hill"
imgs_path = "/content/" + case_name
img_list = sorted(glob.glob(os.path.join(imgs_path, "*.JPG"))) + sorted(glob.glob(os.path.join(imgs_path, "*.jpg")))
print(img_list)


# Try first two images
img1 = cv2.imread(img_list[1])
img2 = cv2.imread(img_list[2])

stitched_12 = stitch_images(img1, img2)

# display stitched image
from google.colab.patches import cv2_imshow
cv2_imshow(stitched_12)

In [ ]:
# Now let's stitch all three images in `hill`.
# Hint: you can stitch the last two images `2.JPG` and `3.JPG`
# and then stitch `1.JPG` with stitched `2.JPG` + `3.JPG`.


In [ ]:
# Now stitch images for `tv`.
# Try some different hyperparameters,
# such as thresh in keypoint match, or error thresh in RANSAC.

### Write-up (15 pt)
*  Describe how to remove incorrect matches with RANSAC
*  Display the best homography H after RANSAC (We recommend normalizing `H` by `H = H / H[2, 2]`)
*  Display the blended images